In [1]:
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
import os.path as path
import time
import glob 
import numpy as np
import pandas as pd
import time
import json
from numpy import savez_compressed
from numpy import load

In [3]:
import os
import sys
sys.path.insert(0, os.path.abspath('./FogDataEDA'))
import FogDataNpzGen as FG
import FogDataloader as dataloader

In [4]:
FogDataConfigs = FG.FogData_Configs(input_path      = None, 
                                     target_path     = None, 
                                     start_date      = FG.year_information['2009'][0], 
                                     finish_date     = FG.year_information['2020'][1], 
                                     data_split_dict = FG.data_split_dict_[0], 
                                     data_straucture = '1D',
                                     lead_time_pred  = 24, 
                                     vis_threshold   = 1, 
                                     points_coords   = FG.NAM_coords,  
                                     predictor_names = FG.NETCDF_PREDICTOR_NAMES['All']).return_config()

In [5]:
train_df, valid_df, test_df = FG.input_dataframe_generater(img_path   = None, 
                                            target_path            = None, 
                                            first_date_string      = FogDataConfigs['start_date'], 
                                            last_date_string       = FogDataConfigs['finish_date'], 
                                            target_binarizing_thre = FogDataConfigs['vis_threshold'], 
                                            year_split_dict        = FogDataConfigs['data_split_dict']).split_data_train_valid_test()


In [6]:
mean_std_dict_name = '/data1/fog/Hamid/SparkMET/EXPs/mean_std.json' 
with open(mean_std_dict_name, 'r') as file:
    norm_mean_std_dict = json.load(file)

In [15]:
train_dataset = dataloader.DataAdopter(train_df, 
                                        map_structure         = '1D', 
                                        predictor_names       = dataloader.NETCDF_PREDICTOR_NAMES['All'], 
                                        lead_time_pred        = 24, 
                                        mean_std_dict         = norm_mean_std_dict,
                                        point_geolocation_dic = dataloader.NAM_coords)

In [ ]:
train_dataset_dict = FG.FogDataNPZ(train_df, 
                            map_structure         = FogDataConfigs['data_straucture'], 
                            predictor_names       = FogDataConfigs['predictor_names'], 
                            lead_time_pred        = FogDataConfigs['lead_time_pred'], 
                            mean_std_dict         = norm_mean_std_dict,
                            point_geolocation_dic = FogDataConfigs['points_coords']).return_data_dict()

In [19]:
train_dataset_dict['20130101_0000']['input'].shape

torch.Size([4, 32, 32, 97])

In [20]:
nam_cube_path = '/data1/fog/Dataset/train_24.npz'
savez_compressed(nam_cube_path, train_dataset_dict)

In [11]:
valid_df.to_csv('/data1/fog/Dataset/valid_2.csv')

In [9]:
valid_dataset_dict = FG.FogDataNPZ(valid_df, 
                            map_structure         = FogDataConfigs['data_straucture'], 
                            predictor_names       = FogDataConfigs['predictor_names'], 
                            lead_time_pred        = FogDataConfigs['lead_time_pred'], 
                            mean_std_dict         = norm_mean_std_dict,
                            point_geolocation_dic = FogDataConfigs['points_coords']).return_data_dict()
valid_nam_cube_path = '/data1/fog/Dataset/valid_24_2.npz'
savez_compressed(valid_nam_cube_path, valid_dataset_dict)

# test_dataset_dict = FG.FogDataNPZ(test_df, 
#                             map_structure         = FogDataConfigs['data_straucture'], 
#                             predictor_names       = FogDataConfigs['predictor_names'], 
#                             lead_time_pred        = FogDataConfigs['lead_time_pred'], 
#                             mean_std_dict         = norm_mean_std_dict,
#                             point_geolocation_dic = FogDataConfigs['points_coords']).return_data_dict()
# test_nam_cube_path = '/data1/fog/Dataset/test_24.npz'
# savez_compressed(test_nam_cube_path, test_dataset_dict)

In [ ]:
train_dict = load('/data1/fog/Dataset/train_24.npz', allow_pickle=True,)#
train_dict['arr_0'][0]

In [ ]:
dict_ = train_dict['arr_0'].tolist()
dict_

In [14]:
dict_['20130101_0000']['input'].shape

torch.Size([4, 32, 32, 97])